### 1. Defult setting

In [ ]:
from google.colab import drive
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt 
from datetime import timedelta
from calendar import monthrange


# % matplotlib inline
# pd.set_option('display.max_columns', None, 'display.max_rows', 20)


# drive.mount('/content/gdrive')
# PATH = 'gdrive/Shared drives/109  統計&DL 期末專案/Dataset/Concat_Behavior_Data'
# os.chdir(PATH)

member_feature = pd.read_csv('member_feature.csv', header=0)

member = pd.read_csv('MemberData.csv')
all_behav = pd.read_csv('total_behavior_all.csv')

**PreProcessing**


In [ ]:
## all_behav 目前只有到 2018/6/26 - 2020/4/30 月的資料
all_behav['ht'] = pd.to_datetime(all_behav['ht'].apply(lambda x: int(x/1000)),unit='s')
all_behav = all_behav[~all_behav.duplicated(['uid','ht','bh'])]
all_behav = all_behav.sort_values(by='ht')


search = pd.read_csv('search.csv')
search['ht'] = pd.to_datetime(search['ht'])
product_view = pd.read_csv('productPageView.csv')
product_view['ht'] = pd.to_datetime(product_view['ht'])
order = pd.read_csv('OrderData.csv')
order['TradesDateTime'] = pd.to_datetime(order['TradesDateTime'])

activity_type = ['userRegisteration','mainPageView','productPageView',\
                 'categoryPageView','activityPageView','search',\
                 'addToCart','checkout','purchase','trafficSource']

## Full Model

In [ ]:
### functions


def getDF_SpecificTimeRange(df,start_year,start_month = None, end_month=None):  
  end_year = start_year 

  if end_month // 13 >= 1:
    end_month = end_month % 12
    end_year = start_year + 1
  
  if start_month == None:
    end = datetime(end_year,end_month,1)
    mask = df['ht'] < end 
    
    return df[mask]

  elif start_month <= 0:
    start = datetime(start_year-1, 12 + start_month,1)
    end = datetime(end_year,end_month,1)
    mask = (df['ht'] > start) & (df['ht'] < end)
    return df[mask]

  elif end_month == None:
    start = datetime(start_year,start_month,1)
    mask = df['ht'] > start
    return df[mask]

  else :
    start = datetime(start_year,start_month,1)
    end =datetime(end_year,end_month,1)    
    mask = (df['ht'] > start) & (df['ht'] < end)
    return df[mask]


month_dict_2019 = dict(zip([i for i in range(1,13)],[monthrange(2019,i)[1] for i in range(1,13)]))

def getDF_SpecificTimeRange2(df,year,month,day,directions=None,delta=0):
  if directions == 'forward':
    start = datetime(year,month,day)
    mask = df['ht'] > start
    return df[mask]
  elif directions == 'backward':
    end = datetime(year,month,day)
    mask = df['ht'] < end
    return df[mask]
  else :
    if delta > 0 :
      start = datetime(year,month,1)
      end = start + timedelta(days = delta)
      mask = (df['ht']>start) & (df['ht']<end)
      return df[mask]
    else :
      end = datetime(year,month,1)
      start = end + timedelta(days = delta)
      mask = (df['ht']>start) & (df['ht']<end)
      return df[mask]


def mergeFeature_tocsv(member_feature, df, old_col_name, new_col_name):
  member_feature = pd.merge(member_feature, df, on='uid', how='left')
  member_feature = member_feature.rename(columns={old_col_name:new_col_name})
  member_feature.to_csv('member_feature.csv',index=False)


def get_CAI(uids):
    end_date = datetime(predict_year,predict_month,1)
    for uid in uids:
        ### 取得uid的個別行為資料
        group = groups.get_group(uid)
        puchase_time = group.loc[group['bh'] == 'purchase','ht'].append(pd.Series(end_date)) 
        interval_time = (puchase_time - puchase_time.shift()).dt.days.values
        total_sum = 0 
        total_weight = 0 

        ### 計算加權總和購買期間
        for i in range(1,len(interval_time)):
    #         print(i,interval_time[i])
            total_weight += i
            total_sum += i*interval_time[i]

        ### 計算加權平均購買期間 
        purchase_range = (end_date - puchase_time.iloc[0]).days    
        weighted_interval = total_sum / total_weight
        average_interval = purchase_range / (len(interval_time)-1)
        CAI = (average_interval - weighted_interval) / average_interval

        ### 加入字典
        CAI_dict[uid] = CAI
    return CAI_dict


def get_concentration(behavior,behav_month,concentration_range,df):    
  
    # create dataframe 
    df = df.loc[df['bh'] == behavior]
    df_con_interval = getDF_SpecificTimeRange(df,behav_year,start_month = predict_month - concentration_range,end_month = predict_month)
    df_behav_interval = getDF_SpecificTimeRange2(df,behav_year,behav_month,month_dict[behav_month],delta=-days)

    # count behavior for each customer 
    interval_count = df_con_interval.groupby('uid').count()['bh'].reset_index().rename({'bh':'interval'},axis=1)
    target_count = df_behav_interval.groupby('uid').count()['bh'].reset_index().rename({'bh':'target'},axis=1)

    # merge to columns
    output = pd.merge(interval_count,target_count,on='uid',how='left')
    # caculate concentration rate
    output['target'] = output['target'].fillna(0)
    output[''+'con_'+behavior] = output[['target','interval']].apply(get_rate,axis=1)
    return output[['uid',''+'con_'+behavior]]


def get_rate(x):
    target = x[0]
    interval = x[1]
    average_count = interval / concentration_range
    if target != interval and interval != 0 :
    return target/average_count
    else :
    return 0


def get_df_concentration(bh_list):
    df_concentration = pd.DataFrame()
    i = 0
    for bh in bh_list:
    if i == 0 :
      df_concentration = get_concentration(bh,behav_month,concentration_range,con_behav)
    else:
      concentration_iter = get_concentration(bh,behav_month,concentration_range,con_behav)
      df_concentration = pd.merge(df_concentration,concentration_iter,on='uid',how='outer')
    i = 1 
    df_concentration = df_concentration.fillna(0)
    return df_concentration


def output_csv(df,behav_year,behav_month,days):
    filename = 'mem_in_{}_{}_behav_{}'.format(behav_year,behav_month,days)
    member_data.to_csv('{}.csv'.format(filename),index=False)


In [ ]:
### run data


month_dict_2019 = dict(zip([i for i in range(1,13)],[monthrange(2019,i)[1] for i in range(1,13)]))

behav_year = 2019
predict_year = 2019
behav_month_list = [2,3,4,5,6,7,8,9,10,11,12]
concentration_range = 3
behav_days = [7,14,21,30]

for days in behav_days:
    for behav_month in behav_month_list:
        predict_year = 2019
        predict_month = behav_month + 1
        print('behav_month',behav_month,'predict_month',predict_month)
        if predict_month > 12 : 
            predict_month = predict_month // 12
            predict_year += 1
        print('behav_month',behav_month,'predict_month',predict_month)
        # create dataframe for all features 
        member_data = pd.DataFrame()

        print(f'行為時間 = {behav_year} 年 {behav_month} 月\n預測時間 = {predict_year} 年 {predict_month} 月\n集中度計算 = {concentration_range} 個月\n行為區間={days}')
        print('--------------------------------')
        # Member feature
        # select features
        features = ['MemberID','Age','RegisterSourceTypeDef','Gender','IsAppInstalled','IsEnableEmail','IsEnablePushNotification','IsEnableShortMessage','MemberCardLevel']

        # create features
        ### age 
        member['Birthday'] = pd.to_datetime(member['Birthday'], format='%Y-%m-%d',errors='coerce')
        member['Age'] = member['Birthday'].apply(lambda x : datetime.today().year - x.year)

        # concat data 
        member_data = pd.concat([member_data,member[features]]).rename(columns={'MemberID':'uid'})
        # ---------------------------------------------------------------------------------------------------#

        # CIA 
        # groupby 
        purchase = all_behav.loc[all_behav['bh'] == 'purchase',]
        purchase = getDF_SpecificTimeRange(purchase,predict_year,None, predict_month).reset_index(drop=True)
        # purchase = purchase[purchase['ht'] < datetime(behav_year,predict_month,1)].reset_index(drop=True)

        groups = purchase.groupby('uid')
        groups_count = groups.count()

        # subtract member with less than 2 puchases 
        groups_over_2 = groups_count.loc[groups_count['bh'] > 1].reset_index()
        uids = groups_over_2['uid'].unique()

        CAI_dict = {}
        CAI_dict = get_CAI(uids)
        df_CAI = pd.DataFrame.from_dict(CAI_dict,orient='index',columns=['CAI']).reset_index().rename(columns={'index':'uid'})
        member_data = pd.merge(member_data,df_CAI,on='uid',how='left')
        print('CAI finished')

        # ---------------------------------------------------------------------------------------------------#

        ### remove duplicated values and invalid status
        order = order[~order.duplicated(['TradesGroupCode','TradesDateTime'])]
        order = order[order['Status'].isin(['Finish','Shipping','New'])]

        ### to datetime
        time_mask = order['TradesDateTime'] <= datetime(predict_year,predict_month,1) #### time 
        order = order[time_mask]

        ### rfm
        rfm = pd.DataFrame(index = order['MemberID'].unique())
        order = order.sort_values(by='TradesDateTime',ascending = False)

        ### freq
        frequency = order.groupby('MemberID')['TsCount'].aggregate('sum')
        frequency = pd.DataFrame(pd.qcut(frequency,7,duplicates='drop',labels=[1,2,3,4])).rename(columns={'TsCount':'frequency'})

        ### recency
        recency = order.groupby('MemberID').nth(0)[['TradesDateTime']]
        recency['recency'] = recency['TradesDateTime'].apply(lambda x : (datetime(behav_year,predict_month,1)- x).days)
        recency = recency.drop('TradesDateTime',axis=1)
        recency = pd.DataFrame(pd.qcut(recency['recency'],5,labels=[1,2,3,4,5]))

        ### monetary 
        monetary = order.groupby('MemberID')['TotalSalesAmount'].aggregate('sum')
        monetary = pd.DataFrame(pd.qcut(monetary,5,duplicates='drop',labels=[5,4,3,2,1])).rename(columns={'TotalSalesAmount':'monetary'})

        ### concat all to rfm
        rfm = pd.concat([rfm,recency,frequency,monetary],axis=1).reset_index().rename(columns={'index':'uid'})

        ### merge to member_data
        member_data = pd.merge(member_data,rfm,on='uid',how='left')
        print('RFM finished')

        # ---------------------------------------------------------------------------------------------------#
        # 目標月份行為
        target_month_behav = getDF_SpecificTimeRange2(all_behav,behav_year,behav_month,month_dict_2019[behav_month],delta = -days)  


        # 預測月份行為
        ### 一個禮拜 / 兩個禮拜 / 一個月
        predict_1week_behav = getDF_InterTimeRange(all_behav,behav_year,behav_month,7)
        predict_2week_behav = getDF_InterTimeRange(all_behav,behav_year,behav_month,14)
        predict_1month_behav = getDF_SpecificTimeRange(all_behav,predict_year,predict_month,predict_month+1) 

        # 行為月份是否購買
        df_target_purchase = target_month_behav.pivot_table(index='uid',values='bh',aggfunc=lambda x : 1 if 'purchase' in x.values else 0).reset_index().rename(columns={'bh':'target_perchase'})
        member_data = pd.merge(member_data,df_target_purchase,on='uid',how='left')

        # 預測月份是否購買
        ### 一個禮拜 / 兩個禮拜 / 一個月
        df_1week_purchase = predict_1week_behav.pivot_table(index='uid',values='bh',aggfunc=lambda x : 1 if 'purchase' in x.values else 0).reset_index().rename(columns={'bh':'1week_purchase'})
        df_2week_purchase = predict_2week_behav.pivot_table(index='uid',values='bh',aggfunc=lambda x : 1 if 'purchase' in x.values else 0).reset_index().rename(columns={'bh':'2week_purchase'})
        df_1month_purchase = predict_1month_behav.pivot_table(index='uid',values='bh',aggfunc=lambda x : 1 if 'purchase' in x.values else 0).reset_index().rename(columns={'bh':'1month_purchase'})
        member_data = pd.merge(member_data,df_1week_purchase,on='uid',how='left')
        member_data = pd.merge(member_data,df_2week_purchase,on='uid',how='left')
        member_data = pd.merge(member_data,df_1month_purchase,on='uid',how='left')

        print('if purhcase finished')


        # 總行為次數
        df_total_behav_count = target_month_behav.pivot_table(index=['uid'],values='bh',aggfunc='count',fill_value=0).reset_index()
        member_data = pd.merge(member_data,df_total_behav_count,on='uid',how='left')

        # 個別行為次數
        df_all_behav = target_month_behav.pivot_table(index=['uid'],columns='bh',aggfunc='count',fill_value=0).reset_index()
        # print('behav columns',df_all_behav.columns)
        idx = pd.IndexSlice
        df_all_behav.columns = df_all_behav.columns.droplevel(level=0)
        df_all_behav = df_all_behav.rename(columns={'':'uid'})
        member_data = pd.merge(member_data,df_all_behav,on='uid',how='left')

        # 總行為天數
        target_month_behav['date'] = target_month_behav['ht'].apply(lambda x : x.date())
        df_visit_day = target_month_behav.pivot_table(index=['uid'],values='date',aggfunc = lambda x : len(x.unique()),fill_value=0).rename(columns={'date':'visit_days'})
        member_data = pd.merge(member_data,df_visit_day,on='uid',how='left')


        # 平均搜尋字數
        search['ht'] = pd.to_datetime(search['ht'])
        target_month_search = getDF_SpecificTimeRange2(search,behav_year,behav_month,month_dict_2019[behav_month],delta=-days)
        target_month_search['length_of_search'] = target_month_search['pr'].apply(lambda x : len(str(x)))
        df_target_month_search = target_month_search.pivot_table(index='uid',values='length_of_search',aggfunc='mean').reset_index().rename(columns={'length_of_search':'mean_of_search'})
        member_data = pd.merge(member_data,df_target_month_search,on='uid',how='left')

        # 搜尋集中度
        target_month_product_view = getDF_SpecificTimeRange2(product_view,behav_year,behav_month,month_dict_2019[behav_month],delta=-days)
        df_target_month_product_view = target_month_product_view.pivot_table(index='uid',values='pr1id',aggfunc= lambda x : x.nunique()).reset_index().rename(columns={'pr1id':'nunique_productview'})
        member_data = pd.merge(member_data,df_target_month_product_view,on='uid',how='left')

        print('before concentration finished')

        # # 行為集中度

        con_behav = getDF_SpecificTimeRange(all_behav,behav_year,predict_month - concentration_range,predict_month)    ## time 
        bh_list = ['viewcategory', 'viewproduct', 'traffic', 'viewmainpage','viewactivity', 'add', 'checkout', 'search', 'purchase','register']

        df_concentration  = get_df_concentration(bh_list)
        member_data = pd.merge(member_data,df_concentration,on='uid',how='left')

        print('concentration finished')
        # attention of product 
        member_data['attention_of_product'] =   member_data['nunique_productview'] / member_data['viewproduct']

        # cart_porpotion
        member_data['cart_porpotion'] =  member_data['add'] / member_data['bh']

        # behav_per_day 
        member_data['behav_per_day'] =  member_data['bh'] / member_data['visit_days']


        # replace abnormal age values
        member_data.loc[member_data['Age'] > 90,'Age'] = np.nan
        member_data.loc[member_data['Age'].isna(),'Age'] = member_data['Age'].mean()

        # output to csv 
        output_csv(member_data,behav_year,behav_month,days)


behav_month 2 predict_month 3
behav_month 2 predict_month 3
行為時間 = 2019 年 2 月
預測時間 = 2019 年 3 月
集中度計算 = 3 個月
行為區間=7
--------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in double_scalars


CAI finished
RFM finished
if purhcase finished


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


before concentration finished
concentration finished
behav_month 3 predict_month 4
behav_month 3 predict_month 4
行為時間 = 2019 年 3 月
預測時間 = 2019 年 4 月
集中度計算 = 3 個月
行為區間=7
--------------------------------


...

before concentration finished
concentration finished
behav_month 12 predict_month 13
behav_month 12 predict_month 1
行為時間 = 2019 年 12 月
預測時間 = 2020 年 1 月
集中度計算 = 3 個月
行為區間=30
--------------------------------


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: RuntimeWarning: invalid value encountered in double_scalars


CAI finished
RFM finished
if purhcase finished


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


before concentration finished
concentration finished


# 2. Features -sperated 

會員資料：Age / Membercardlevel / Gender  <br>
會員特性：RFM / CAI / Behavior per day / concentration rate <br>
會員行為：attention of product / average search / porpotion of chart  <br>

In [ ]:
# create dataframe for all features 
member_data = pd.DataFrame()

# define times 
behav_year = 2019
behav_month = 7
predict_month = behav_month + 1
concentration_range = 6

# remove bh out of date
all_behav = all_behav[all_behav['ht'] < datetime(behav_year,predict_month + 1,1) ]

print(f'行為時間 = {behav_year} 年 {behav_month} 月\n預測時間 = {behav_year} 年 {behav_month} 月\n集中度計算 = {concentration_range} 個月')

行為時間 = 2019 年 7 月
預測時間 = 2019 年 7 月
集中度計算 = 6 個月


### a. 會員資料

In [ ]:
# Member feature

# select features
features = ['MemberID','Age','RegisterSourceTypeDef','Gender','IsAppInstalled','IsEnableEmail','IsEnablePushNotification','IsEnableShortMessage','MemberCardLevel']

# create features
### age 
member['Birthday'] = pd.to_datetime(member['Birthday'], format='%Y-%m-%d',errors='coerce')
member['Age'] = member['Birthday'].apply(lambda x : datetime.today().year - x.year)

# concat data 
member_data = pd.concat([member_data,member[features]]).rename(columns={'MemberID':'uid'})
member_data.head()

,uid,Age,RegisterSourceTypeDef,Gender,IsAppInstalled,IsEnableEmail,IsEnablePushNotification,IsEnableShortMessage,MemberCardLevel
0,b%2Be6RCNYUwTRO%2BnSBeE4H7ySets9zgOaGkqDYItQZS...,37.0,iOSApp,Female,True,True,True,True,10
1,%2FL156mdTEuzDIF5ulSFL4mhJdNRGs8haMhCgz%2BSrG9...,30.0,NaN,Female,True,True,True,True,10
2,wj5w28khA7PlMGGXYaUHIfTxKhjae8pnB76k46OL%2Bhc%3D,120.0,NaN,NaN,True,True,True,True,10
3,rvJ4K4pF5BYUVUVZnRdcsc0AMlVLX0T2warN0CyPKmU%3D,43.0,NaN,Female,True,True,True,True,10
4,Oalq5zcQazy%2B%2BnozogEXMbTItDObkJMUPzz4H%2Fn7...,37.0,NaN,Female,True,True,True,True,10


### b. 會員特性

**CAI**

In [ ]:
# groupby 
purchase = all_behav.loc[all_behav['bh'] == 'purchase',]
purchase = purchase[purchase['ht'] < datetime(behav_year,predict_month,1)].reset_index(drop=True)
groups = purchase.groupby('uid')
groups_count = groups.count()

# subtract member with less than 2 puchases 
groups_over_2 = groups_count.loc[groups_count['bh'] > 1].reset_index()
uids = groups_over_2['uid'].unique()



CAI_dict = {}

def get_CAI(uids):
    end_date = datetime(behav_year,behav_month+1,1)
    for uid in uids:
        ### 取得uid的個別行為資料
        group = groups.get_group(uid)
        puchase_time = group.loc[group['bh'] == 'purchase','ht'].append(pd.Series(end_date)) 
        interval_time = (puchase_time - puchase_time.shift()).dt.days.values
        total_sum = 0 
        total_weight = 0 

        ### 計算加權總和購買期間
        for i in range(1,len(interval_time)):
    #         print(i,interval_time[i])
            total_weight += i
            total_sum += i*interval_time[i]

        ### 計算加權平均購買期間 
        purchase_range = (end_date - puchase_time.iloc[0]).days    
        weighted_interval = total_sum / total_weight
        average_interval = purchase_range / (len(interval_time)-1)
        CAI = (average_interval - weighted_interval) / average_interval

        ### 加入字典
        CAI_dict[uid] = CAI
    return CAI_dict

CAI_dict = get_CAI(uids)
df_CAI = pd.DataFrame.from_dict(CAI_dict,orient='index',columns=['CAI']).reset_index().rename(columns={'index':'uid'})
member_data = pd.merge(member_data,df_CAI,on='uid',how='left')
member_data.head()

# # # plotting
# plt.hist(df_CAI['CAI'],bins=30)
# plt.show()

# 30 secs

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars


,uid,Age,RegisterSourceTypeDef,Gender,IsAppInstalled,IsEnableEmail,IsEnablePushNotification,IsEnableShortMessage,MemberCardLevel,CAI
0,b%2Be6RCNYUwTRO%2BnSBeE4H7ySets9zgOaGkqDYItQZS...,37.0,iOSApp,Female,True,True,True,True,10,NaN
1,%2FL156mdTEuzDIF5ulSFL4mhJdNRGs8haMhCgz%2BSrG9...,30.0,NaN,Female,True,True,True,True,10,NaN
2,wj5w28khA7PlMGGXYaUHIfTxKhjae8pnB76k46OL%2Bhc%3D,120.0,NaN,NaN,True,True,True,True,10,NaN
3,rvJ4K4pF5BYUVUVZnRdcsc0AMlVLX0T2warN0CyPKmU%3D,43.0,NaN,Female,True,True,True,True,10,NaN
4,Oalq5zcQazy%2B%2BnozogEXMbTItDObkJMUPzz4H%2Fn7...,37.0,NaN,Female,True,True,True,True,10,NaN


**RFM**

In [ ]:
### remove duplicated values and invalid status
order = order[~order.duplicated(['TradesGroupCode','TradesDateTime'])]
order = order[order['Status'].isin(['Finish','Shipping','New'])]

### to datetime
order['TradesDateTime'] = pd.to_datetime(order['TradesDateTime'])
time_mask = order['TradesDateTime'] <= datetime(behav_year,behav_month+1,1)
order = order[time_mask]

### rfm
rfm = pd.DataFrame(index = order['MemberID'].unique())
order = order.sort_values(by='TradesDateTime',ascending = False)

### freq
frequency = order.groupby('MemberID')['TsCount'].aggregate('sum')
frequency = pd.DataFrame(pd.qcut(frequency,7,duplicates='drop',labels=[1,2,3,4])).rename(columns={'TsCount':'frequency'})

### recency
recency = order.groupby('MemberID').nth(0)[['TradesDateTime']]
recency['recency'] = recency['TradesDateTime'].apply(lambda x : (datetime(behav_year,predict_month,1)- x).days)
recency = recency.drop('TradesDateTime',axis=1)
recency = pd.DataFrame(pd.qcut(recency['recency'],5,labels=[1,2,3,4,5]))

### monetary 
monetary = order.groupby('MemberID')['TotalSalesAmount'].aggregate('sum')
monetary = pd.DataFrame(pd.qcut(monetary,5,duplicates='drop',labels=[5,4,3,2,1])).rename(columns={'TotalSalesAmount':'monetary'})

### concat all to rfm
rfm = pd.concat([rfm,recency,frequency,monetary],axis=1).reset_index().rename(columns={'index':'uid'})

### merge to member_data
member_data = pd.merge(member_data,rfm,on='uid',how='left')
member_data.head()

,uid,Age,RegisterSourceTypeDef,Gender,IsAppInstalled,IsEnableEmail,IsEnablePushNotification,IsEnableShortMessage,MemberCardLevel,CAI,recency,frequency,monetary
0,b%2Be6RCNYUwTRO%2BnSBeE4H7ySets9zgOaGkqDYItQZS...,37.0,iOSApp,Female,True,True,True,True,10,NaN,3,3,1
1,%2FL156mdTEuzDIF5ulSFL4mhJdNRGs8haMhCgz%2BSrG9...,30.0,NaN,Female,True,True,True,True,10,NaN,NaN,NaN,NaN
2,wj5w28khA7PlMGGXYaUHIfTxKhjae8pnB76k46OL%2Bhc%3D,120.0,NaN,NaN,True,True,True,True,10,NaN,5,1,5
3,rvJ4K4pF5BYUVUVZnRdcsc0AMlVLX0T2warN0CyPKmU%3D,43.0,NaN,Female,True,True,True,True,10,NaN,NaN,NaN,NaN
4,Oalq5zcQazy%2B%2BnozogEXMbTItDObkJMUPzz4H%2Fn7...,37.0,NaN,Female,True,True,True,True,10,NaN,NaN,NaN,NaN


### c . 會員行為

attention of product / average search / porpotion of chart / target_purchase / 
predict_purchase

In [ ]:
# 目標月份行為
target_month_behav = getDF_SpecificTimeRange(behav_year,behav_month,behav_month+1,all_behav)

# 預測月份行為
predict_month_behav = getDF_SpecificTimeRange(behav_year,predict_month,predict_month+1,all_behav)

# 總行為次數
df_total_behav_count = target_month_behav.pivot_table(index=['uid'],values='bh',aggfunc='count',fill_value=0).reset_index()
member_data = pd.merge(member_data,df_total_behav_count,on='uid',how='left')

# 個別行為次數
df_all_behav = target_month_behav.pivot_table(index=['uid'],columns='bh',aggfunc='count',fill_value=0).reset_index()
idx = pd.IndexSlice
df_all_behav.columns = df_all_behav.columns.droplevel(level=0)
df_all_behav = df_all_behav.rename(columns={'':'uid'})
member_data = pd.merge(member_data,df_all_behav,on='uid',how='left')

# 總行為天數

target_month_behav['date'] = target_month_behav['ht'].apply(lambda x : x.date())
df_visit_day = target_month_behav.pivot_table(index=['uid'],values='date',aggfunc = lambda x : len(x.unique()),fill_value=0).rename(columns={'date':'visit_days'})
member_data = pd.merge(member_data,df_visit_day,on='uid',how='left')

# 目標月份是否購買
df_target_purchase = target_month_behav.pivot_table(index='uid',values='bh',aggfunc=lambda x : 1 if 'purchase' in x.values else 0).reset_index().rename(columns={'bh':'target_purchase'})
member_data = pd.merge(member_data,df_target_purchase,on='uid',how='left')

# 預測月份是否購買
df_predict_purchase = predict_month_behav.pivot_table(index='uid',values='bh',aggfunc=lambda x : 1 if 'purchase' in x.values else 0).reset_index().rename(columns={'bh':'predict_purchase'})
member_data = pd.merge(member_data,df_predict_purchase,on='uid',how='left')

# 平均搜尋字數
search = pd.read_csv('search.csv')
search['ht'] = pd.to_datetime(search['ht'])
target_month_search = getDF_SpecificTimeRange(behav_year,behav_month,behav_month+1,search)
target_month_search['length_of_search'] = target_month_search['pr'].apply(lambda x : len(str(x)))
df_target_month_search = target_month_search.pivot_table(index='uid',values='length_of_search',aggfunc='mean').reset_index().rename(columns={'length_of_search':'mean_of_search'})
member_data = pd.merge(member_data,df_target_month_search,on='uid',how='left')

# 搜尋集中度

product_view = pd.read_csv('productPageView.csv')
product_view['ht'] = pd.to_datetime(product_view['ht'])
target_month_product_view = getDF_SpecificTimeRange(behav_year,behav_month,behav_month+1,product_view)
df_target_month_product_view = target_month_product_view.pivot_table(index='uid',values='pr1id',aggfunc= lambda x : x.nunique()).reset_index().rename(columns={'pr1id':'nunique_productview'})
member_data = pd.merge(member_data,df_target_month_product_view,on='uid',how='left')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# # 行為集中度

con_month = predict_month - concentration_range # 從幾月開始算集中度
start_month = behav_month # 從幾月開始算行為數
con_behav = getDF_SpecificTimeRange(behav_year,predict_month - concentration_range,predict_month,all_behav)

def get_concentration(behavior,con_year,con_month,behav_year,behav_month,df):
  df = df.loc[df['bh'] == behavior]
  # create mask for date 
  con_interval = (df['ht'] > datetime(con_year,con_month,1))
  target_interval = (df['ht'] > datetime(behav_year,behav_month,1))
  # create dataframe - count behavior for each customer 
  interval = df[con_interval]
  target = df[target_interval]
  
  interval_count = interval.groupby('uid').count()['bh'].reset_index().rename({'bh':'interval'},axis=1)
  target_count = target.groupby('uid').count()['bh'].reset_index().rename({'bh':'target'},axis=1)

  # merge to columns
  output = pd.merge(interval_count,target_count,on='uid',how='left')
  # caculate concentration rate
  output['target'] = output['target'].fillna(0)
  output[''+'con_'+behavior] = output[['target','interval']].apply(get_rate,axis=1)
  return output[['uid',''+'con_'+behavior]]
  # return output.drop(['target','interval'],axis=1)

def get_rate(x):
  target = x[0]
  interval = x[1]
  average_count = interval / concentration_range
  if target != interval and interval != 0 :
    return target/average_count
  else :
    return 0

bh_list = ['viewcategory', 'viewproduct', 'traffic', 'viewmainpage','viewactivity', 'add', 'checkout', 'search', 'purchase','register']

def get_df_concentration(bh_list):
  df_concentration = pd.DataFrame()
  i = 0
  for bh in bh_list:
    if i == 0 :
      df_concentration = get_concentration(bh,con_month,start_month,con_behav)
    else:
      concentration_iter = get_concentration(bh,con_month,start_month,con_behav)
      df_concentration = pd.merge(df_concentration,concentration_iter,on='uid',how='outer')
    i = 1 
  df_concentration = df_concentration.fillna(0)
  return df_concentration

df_concentration  = get_df_concentration(bh_list)

member_data = pd.merge(member_data,df_concentration,on='uid',how='left')
member_data

,uid,Age,RegisterSourceTypeDef,Gender,IsAppInstalled,IsEnableEmail,IsEnablePushNotification,IsEnableShortMessage,CAI,recency,frequency,monetary,bh,add,checkout,purchase,register,search,traffic,viewactivity,viewcategory,viewmainpage,viewproduct,visit_days,predict_purchase,length_of_search,nunique_search,target_purchase,con_viewcategory,con_viewproduct,con_traffic,con_viewmainpage,con_viewactivity,con_add,con_checkout,con_search,con_purchase,con_register
0,b%2Be6RCNYUwTRO%2BnSBeE4H7ySets9zgOaGkqDYItQZS...,37.0,iOSApp,Female,True,True,True,True,NaN,3,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,%2FL156mdTEuzDIF5ulSFL4mhJdNRGs8haMhCgz%2BSrG9...,30.0,NaN,Female,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wj5w28khA7PlMGGXYaUHIfTxKhjae8pnB76k46OL%2Bhc%3D,120.0,NaN,NaN,True,True,True,True,NaN,5,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rvJ4K4pF5BYUVUVZnRdcsc0AMlVLX0T2warN0CyPKmU%3D,43.0,NaN,Female,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Oalq5zcQazy%2B%2BnozogEXMbTItDObkJMUPzz4H%2Fn7...,37.0,NaN,Female,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337571,6d4pC49pQMvXzqQj9hsf%2F6RF1rj9ESYcChTy%2BAzKED...,120.0,iOSApp,NaN,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337572,xQNG23Ye8atMvzJMlM5I74otpOhGY2BTWepJKlPQip0%3D,120.0,LocationWizard,NaN,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337573,d91zBcNywvaZRiZ4vKFKTLQzEO5Bkw4cXVF4DCr0Tro%3D,40.0,LocationWizard,Female,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337574,sBlp7orYt%2B5z8079wpkEFWSm9h7u1IF%2B4aeP6jeAtj...,44.0,LocationWizard,Female,True,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### d. 建立model所需的欄位

In [ ]:
# attention of product 
member_data['attention_of_product'] =   member_data['nunique_productview'] / member_data['viewproduct']

# cart_porpotion
member_data['cart_porpotion'] =  member_data['add'] / member_data['bh']

# behav_per_day 
member_data['behav_per_day'] =  member_data['bh'] / member_data['visit_days']

In [ ]:
# from sklearn.impute import KNNImputer

member_data.loc[member_data['Age'] > 90,'Age'] = np.nan
member_data.loc[member_data['Age'].isna(),'Age'] = member_data['Age'].mean()


def output_csv(df,behav_year,behav_month):
  filename = 'mem_in_{}_{}'.format(behav_year,behav_month)
  member_data.to_csv('{}.csv'.format(filename),index=False)

output_csv(member_data,behav_year,behav_month)